In [2]:
# filter some warning messages
import warnings
warnings.filterwarnings("ignore") 

In [3]:
# get cm4 so and thetao data ('gn' without regriding)
from intake import open_catalog
url = "https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/master.yaml"
col = open_catalog(url).climate.cmip6_gcs()

dslist = col.search(variable_id = ['so', 'thetao'], table_id = 'Omon', experiment_id = 'piControl', 
           source_id = 'GFDL-CM4', grid_label = 'gn')
ds = dslist['CMIP.NOAA-GFDL.GFDL-CM4.piControl.Omon.gn'].to_dask()
ds = ds.squeeze('member_id').reset_coords('member_id', drop = True)

In [4]:
import numpy as np
import xesmf as xe
import xarray as xr

import copy
import gsw

In [3]:
ds

<xarray.Dataset>
Dimensions:    (bnds: 2, y: 1080, x: 1440, vertex: 4, lev: 35, time: 6000)
Coordinates:
  * bnds       (bnds) float64 1.0 2.0
    lat        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lat_bnds   (y, x, vertex) float32 dask.array<chunksize=(1080, 1440, 4), meta=np.ndarray>
  * lev        (lev) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
    lev_bnds   (lev, bnds) float64 dask.array<chunksize=(35, 2), meta=np.ndarray>
    lon        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lon_bnds   (y, x, vertex) float32 dask.array<chunksize=(1080, 1440, 4), meta=np.ndarray>
  * time       (time) object 0151-01-16 12:00:00 ... 0650-12-16 12:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(6000, 2), meta=np.ndarray>
  * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
Dimensions without coordinates: vertex
Data variables:
    so         (time, lev, y, x) float32 dask.array<chunksize=(2, 35, 1080, 1440), meta=np.ndarray>
    thetao     (time, lev, y, x) float32 dask.array<chunksize=(1, 35, 1080, 1440), meta=np.ndarray>
Attributes: (12/47)
    tracking_id:             hdl:21.14100/f6cc54cd-1d34-4ce8-a560-53a3bbe93a1...
    grid_label:              gn
    history:                 File was processed by fremetar (GFDL analog of C...
    source_type:             AOGCM
    realization_index:       1
    institution:             National Oceanic and Atmospheric Administration,...
    ...                      ...
    parent_source_id:        GFDL-CM4
    comment:                 <null ref>
    grid:                    ocean data on native grid tripolar - nominal 0.2...
    nominal_resolution:      25 km
    parent_experiment_id:    piControl-spinup
    intake_esm_dataset_key:  CMIP.NOAA-GFDL.GFDL-CM4.piControl.Omon.gn

In [13]:
newdep = np.append(ds.lev_bnds.values[:,0], ds.lev_bnds.values[-1,1])
len(newdep)

36

In [14]:
newdep

array([0.000e+00, 5.000e+00, 1.500e+01, 2.500e+01, 4.000e+01, 6.250e+01,
       8.750e+01, 1.125e+02, 1.375e+02, 1.750e+02, 2.250e+02, 2.750e+02,
       3.500e+02, 4.500e+02, 5.500e+02, 6.500e+02, 7.500e+02, 8.500e+02,
       9.500e+02, 1.050e+03, 1.150e+03, 1.250e+03, 1.350e+03, 1.450e+03,
       1.625e+03, 1.875e+03, 2.250e+03, 2.750e+03, 3.250e+03, 3.750e+03,
       4.250e+03, 4.750e+03, 5.250e+03, 5.750e+03, 6.250e+03, 6.750e+03])

In [15]:
ds_news = ds.so.interp(lev = newdep, method='cubic')
ds_newt = ds.thetao.interp(lev = newdep, method='cubic')

<xarray.DataArray 'so' (time: 6000, lev: 36, y: 1080, x: 1440)>
dask.array<transpose, shape=(6000, 36, 1080, 1440), dtype=float32, chunksize=(2, 36, 1080, 1440), chunktype=numpy.ndarray>
Coordinates:
    lat      (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lon      (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
  * time     (time) object 0151-01-16 12:00:00 ... 0650-12-16 12:00:00
  * x        (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.28 59.53 59.78 60.03
  * y        (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.62 89.73 89.84 89.95
  * lev      (lev) float64 0.0 5.0 15.0 25.0 ... 5.75e+03 6.25e+03 6.75e+03
Attributes:
    cell_measures:  area: areacello volume: volcello
    cell_methods:   area: mean where sea time: mean
    long_name:      Sea Water Salinity
    original_name:  so
    standard_name:  sea_water_salinity
    units:          0.001

In [3]:
ds_gswp = gsw.p_from_z(-ds.lev, ds.lat)
ds_pbnds = gsw.p_from_z(-ds.lev_bnds, ds.lat)
ds_sa = gsw.SA_from_SP(ds.so, ds_gswp, ds.lon, ds.lat)
ds_ct = gsw.CT_from_t(ds_sa, ds.thetao, ds_gswp)

In [9]:
ds_pbnds.isel(bnds = -1) - ds_pbnds.isel(bnds = 0)

<xarray.DataArray 'lev_bnds' (lev: 35, y: 1080, x: 1440)>
dask.array<sub, shape=(35, 1080, 1440), dtype=float64, chunksize=(35, 1080, 1440), chunktype=numpy.ndarray>
Coordinates:
  * lev      (lev) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
    lat      (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lon      (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
  * x        (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.28 59.53 59.78 60.03
  * y        (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.62 89.73 89.84 89.95

In [ ]:
### modeified from: https://teos-10.github.io/GSW-Python/_modules/gsw/stability.html#Nsquared
def Nsquared(SA, CT, p, lat, pbnds):
    """
    Calculate the square of the buoyancy frequency.

    Parameters
    ----------
    SA : array-like
        Absolute Salinity, g/kg
    CT : array-like
        Conservative Temperature (ITS-90), degrees C
    p : array-like
        Sea pressure (absolute pressure minus 10.1325 dbar), dbar
    lat : array-like, 1-D, optional
        Latitude, degrees.
    axis : int, optional
        The dimension along which pressure increases.

    Returns
    -------
    N2 : array
        Buoyancy frequency-squared at pressure midpoints, 1/s.
        The shape along the pressure axis dimension is one
        less than that of the inputs.
    p_mid : array
        Pressure at midpoints of p, dbar.
        The array shape matches N2.

    """
    if lat is not None:
        if np.any((lat < -90) | (lat > 90)):
            raise ValueError('lat is out of range')
        # SA, CT, p, lat = np.broadcast_arrays(SA, CT, p, lat)
        g = gsw.grav(lat, p)
    else:
        # SA, CT, p = np.broadcast_arrays(SA, CT, p)
        g = 9.7963  # (Griffies, 2004)

    db_to_pa = 1e4
    # shallow = axis_slicer(SA.ndim, slice(-1), axis)
    # deep = axis_slicer(SA.ndim, slice(1, None), axis)
    # if lat is not None:
    #     g_local = 0.5 * (g[shallow] + g[deep])
    # else:
    #     g_local = g
    g_local = g

    # dSA = SA[deep] - SA[shallow]
    # dCT = CT[deep] - CT[shallow]
    # dp = p[deep] - p[shallow]
    # SA_mid = 0.5 * (SA[shallow] + SA[deep])
    # CT_mid = 0.5 * (CT[shallow] + CT[deep])
    # p_mid = 0.5 * (p[shallow] + p[deep])
    dp = pbnds.isel(bnds = -1) - pbnds.isel(bnds = 0)

    specvol_mid, alpha_mid, beta_mid = gsw.specvol_alpha_beta(SA, CT, p)

    N2 = ((g_local**2) / (specvol_mid * db_to_pa * dp))
    N2 *= (beta_mid*dSA - alpha_mid*dCT)

    return N2, p_mid


In [ ]:
ds_n2 = gsw.Nsquared(ds_sa, ds_ct, ds_gswp, ds.lat)